In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import torch
from torch import nn
from torch import optim
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, models

In [3]:
from torch.autograd import Variable
import torch.utils.data as data
import pandas as pd
import os
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset


In [4]:
# Path for (unlabeled) test data
test_path = '/content/drive/My Drive/PatternProject/Test/'

In [5]:
# Prepare test data set

data =[]
label =[]
resize = transforms.Compose([transforms.CenterCrop(400)]) #crop to 400

files = os.listdir(test_path)
files.sort(); # so that resutls can be printed in the order of 'test01', 'test02', ... 'test80'

print(files)

for f in files:
  img = Image.open(test_path+'/'+f,'r')
  img = resize(img)
  r,g,b = img.split()
  one_img = np.asarray(np.float32(r)/255.0)
  img = np.asarray([one_img])
  data.append(img)
  label.append(1) # dummy numbers (not to be used anyway)

data = np.array(data, dtype='float32')
label = np.array(label, dtype='int64')

test_X = torch.from_numpy(data)
test_Y = torch.from_numpy(label)
test_X = test_X.type(torch.cuda.FloatTensor)
test_Y = test_Y.type(torch.cuda.LongTensor)
test_dataset = TensorDataset(test_X, test_Y)
#test_dataset = TensorDataset(test_X)
test_loader = DataLoader(test_dataset, batch_size = 80, shuffle=False) # set batch size to 80 to print all the predictions in one vector



['test01.jpg', 'test02.jpg', 'test03.jpg', 'test04.jpg', 'test05.jpg', 'test06.jpg', 'test07.jpg', 'test08.jpg', 'test09.jpg', 'test10.jpg', 'test11.jpg', 'test12.jpg', 'test13.jpg', 'test14.jpg', 'test15.jpg', 'test16.jpg', 'test17.jpg', 'test18.jpg', 'test19.jpg', 'test20.jpg', 'test21.jpg', 'test22.jpg', 'test23.jpg', 'test24.jpg', 'test25.jpg', 'test26.jpg', 'test27.jpg', 'test28.jpg', 'test29.jpg', 'test30.jpg', 'test31.jpg', 'test32.jpg', 'test33.jpg', 'test34.jpg', 'test35.jpg', 'test36.jpg', 'test37.jpg', 'test38.jpg', 'test39.jpg', 'test40.jpg', 'test41.jpg', 'test42.jpg', 'test43.jpg', 'test44.jpg', 'test45.jpg', 'test46.jpg', 'test47.jpg', 'test48.jpg', 'test49.jpg', 'test50.jpg', 'test51.jpg', 'test52.jpg', 'test53.jpg', 'test54.jpg', 'test55.jpg', 'test56.jpg', 'test57.jpg', 'test58.jpg', 'test59.jpg', 'test60.jpg', 'test61.jpg', 'test62.jpg', 'test63.jpg', 'test64.jpg', 'test65.jpg', 'test66.jpg', 'test67.jpg', 'test68.jpg', 'test69.jpg', 'test70.jpg', 'test71.jpg', 'test

In [7]:
# Build convolutional neural net
# IMPORTNT: This structure should be same as the one used for training (so simply copy and paste the correponding codes)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(
           
            nn.Conv2d(1, 16, 3), 
            nn.ReLU(),
            nn.Conv2d(16,16, 3), 
            nn.ReLU(),
            nn.MaxPool2d(2,2),  

            nn.Conv2d(16, 32, 3),  
            nn.ReLU(),
            nn.Conv2d(32, 32, 3), 
            nn.ReLU(),
            nn.MaxPool2d(2,2),   

            nn.Conv2d(32, 64, 3), 
            nn.ReLU(),
            nn.Conv2d(64, 64, 3), 
            nn.ReLU(),
            nn.Conv2d(64, 64, 3), 
            nn.ReLU(),
            nn.MaxPool2d(2,2),  

        )
        
        self.fc = nn.Sequential(
            nn.Linear(45*45*64,64),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,2)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size()[0],-1)
        x = self.fc(x)
        return x


model = Net()

In [8]:
model = torch.load('/content/drive/My Drive/PatternProject/Model_JiYe_1770027')

In [9]:
model.eval()
criterion = nn.CrossEntropyLoss()
a = []

#각각 80,10길이의 result로 두개가 나와서 하나의 리스트로 합치는 과정 포함
for inputs, labels in test_loader:
    inputs, labels = Variable(inputs), Variable(labels)
    output = model(inputs)
    idx, pred = output.max(1)
    pred = (pred.detach().cpu().numpy())
    print('Test results : {}'.format(pred))
    b = pred.tolist()
    a.append(b)


Test results : [0 0 1 0 0 1 0 1 1 0 0 1 1 0 1 1 1 0 1 1 1 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1
 0 1 1 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 0
 0 0 0 1 0 0]
Test results : [0 0 1 1 0 0 0 0 0 0]


In [10]:
#2차원 리스트로 되어있어서 하나의 리스트로 변환
final_result = []
for i in range(80):
  final_result.append(a[0][i])

for j in range(10):
  final_result.append(a[1][j])

print(type(final_result), len(final_result))
print(final_result)

<class 'list'> 90
[0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]


In [11]:
import csv 
import pandas as pd
data = np.array(files)
data.T
z1 =  data.tolist()

data2 =  np.array(final_result)
data2.T
z2 = data2.tolist()
    
z3 = pd.DataFrame(z1)
z4 = pd.DataFrame(z2)
z5= pd.concat([z3,z4],axis=1)
z5
z5.to_csv("/content/drive/My Drive/PatternProject/Result_JiYe_1770027.csv", mode='w',header=False,index = None)